In [11]:
import cv2
print('opencv: ' +  cv2.__version__)

import numpy as np
import math

import pyrealsense2 as rs

from Odometry2D import VisualOdometry
from PinholeCamera import PinholeCamera
from OdometryUtil import truncate

opencv: 3.4.3


### Camera settings

In [12]:
InfWidth = 1280
InfHeight = 720
InfFPS = 30

RES_W = InfWidth
RES_H = InfHeight
FOCAL = 1.88
PRI_W = RES_W / 2
PRI_H = RES_H / 2

cam = PinholeCamera(RES_W, RES_H, FOCAL, FOCAL, PRI_W, PRI_H)

deviceDetect = False
ctx = rs.context()
devices = ctx.query_devices()

for dev in devices:
    productName = str(dev.get_info(rs.camera_info.product_id))
    print(productName)
  
    # DS 435 config
    if productName in "0B07":
        # EnableColorStream = True
        EnableInfraredStream = True
        # EnableDepthStream = True
        deviceDetect = True
        break
    
    # DS 415 config
    elif productName in "0AD3":
        # EnableColorStream = True
        EnableInfraredStream = True
        # EnableDepthStream = True
        deviceDetect = True
        break
        
    # DS 410 config
    elif productName in "0AD2":
        # EnableColorStream = False
        EnableInfraredStream = True
        # EnableDepthStream = True
        deviceDetect = True
        break

if deviceDetect is not True:
    raise Exception("No supported device was found")
    
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableInfraredStream is True:
    config.enable_stream(rs.stream.infrared, 1, InfWidth, InfHeight, rs.format.y8, InfFPS)
    config.enable_stream(rs.stream.infrared, 2, InfWidth, InfHeight, rs.format.y8, InfFPS)

# Start streaming
pipeline.start(config)

0B07


### Output settings

In [13]:
OUT_W = 1000
OUT_H = 1100

START_X = 500
START_Y = 550

traj = np.zeros((OUT_W, OUT_H, 3), dtype=np.uint8)

font = cv2.FONT_HERSHEY_TRIPLEX
fontColor = (255, 255, 0)
trajColor = (0, 255, 255)

### Visual Odometry

In [14]:
vo = VisualOdometry(cam, False)
vo_switch = False
frame_id = 0

try:
    while True:
        time1 = cv2.getTickCount()
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        
        inf_frame1 = frames.get_infrared_frame(1) # Left IR Camera, it allows 0, 1 or no input
        inf_frame2 = frames.get_infrared_frame(2) # Right IR camera
        if not inf_frame1:
            break

        img1 = np.asanyarray(inf_frame1.get_data())
        img2 = np.asanyarray(inf_frame2.get_data())

        if vo_switch:
            imgGray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
            vo.update(imgGray, frame_id)

            cur_t = vo.cur_t
            x, y, z = cur_t[0], cur_t[1], cur_t[2]

            draw_x, draw_y = int(x)+START_X, int(z)+START_Y
            cv2.circle(traj, (draw_x,draw_y), 1, trajColor, 1)

            cv2.rectangle(traj, (0, 0), (OUT_W, 100), (0,0,0), -1)
            text = "Coordinates: x=%2f y=%2f z=%2f"%(truncate(x,2), truncate(y,2), truncate(z,2))
            cv2.putText(traj, text, (10,30), font, 1, fontColor, 1, 8)

        time2 = cv2.getTickCount()
        time = math.floor(((time2 - time1) * 1000) / cv2.getTickFrequency())
        
        imgOut = cv2.resize(img1, (int(RES_W / 2), int(RES_H / 2)), interpolation=cv2.INTER_CUBIC)
        imgOut = cv2.copyMakeBorder(imgOut, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        textString = str(time) + " ms "
        cv2.putText(imgOut, textString, (10, int(RES_H / 2) + 40), font, 1, fontColor, 2, 8)
        cv2.imshow('Left IR Camera', imgOut)
        
        imgOut = cv2.resize(img2, (int(RES_W / 2), int(RES_H / 2)), interpolation=cv2.INTER_CUBIC)
        cv2.imshow('Right IR Camera', imgOut)
        cv2.imshow('Trajectory', traj)

        getKey = cv2.waitKey(10) & 0xFF
        if getKey == ord('q'):
            break
        elif getKey == ord('a'):
            vo_switch = True
            print('Visual odometry start')

except Exception as e:
    print(e)
    pass

finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipeline.stop()